In [2]:
import nltk
import sys

TERMINALS = """
Adj -> "country" | "dreadful" | "enigmatical" | "little" | "moist" | "red"
Adv -> "down" | "here" | "never"
Conj -> "and" | "until"
Det -> "a" | "an" | "his" | "my" | "the"
N -> "armchair" | "companion" | "day" | "door" | "hand" | "he" | "himself"
N -> "holmes" | "home" | "i" | "mess" | "paint" | "palm" | "pipe" | "she"
N -> "smile" | "thursday" | "walk" | "we" | "word"
P -> "at" | "before" | "in" | "of" | "on" | "to"
V -> "arrived" | "came" | "chuckled" | "had" | "lit" | "said" | "sat"
V -> "smiled" | "tell" | "were"
"""

NONTERMINALS = """
S -> NP VP | VP NP | S Conj S
AdjP -> Adj | Adj AdjP
PP -> P NP
NP -> N | Det N | Det AdjP N | NP PP
VP -> V | Adv VP | VP Adv | V NP | NP V |VP PP

"""

grammar = nltk.CFG.fromstring(NONTERMINALS + TERMINALS)
parser = nltk.ChartParser(grammar)

In [74]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anuj_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def preprocess(sentence):
    """
    Convert `sentence` to a list of its words.
    Pre-process sentence by converting all characters to lowercase
    and removing any word that does not contain at least one alphabetic
    character.
    """
    words = nltk.word_tokenize(sentence)
    words = [word.lower() for word in words
             if any(char.isalpha() for char in word)]
    return words

In [53]:
def np_chunk(tree):
    """
    Return a list of all noun phrase chunks in the sentence tree.
    A noun phrase chunk is defined as any subtree of the sentence
    whose label is "NP" that does not itself contain any other
    noun phrases as subtrees.
    """
    noun_phrases = []
    for subtree in tree.subtrees(lambda x: x.label() == "NP"):
        if len(list(subtree.subtrees(lambda x: x.label() == "NP"))) == 1:
            noun_phrases.append(subtree)
    return noun_phrases

In [260]:
import os
with os.scandir("sentences") as sentences:
    for sentence in sentences:
        with open(sentence.path) as f:
            s = f.read()
            print(s)
            treer(preprocess(s))
            print("\n\n")

Holmes sat.

        S     
   _____|___   
  NP        VP
  |         |  
  N         V 
  |         |  
holmes     sat




I had a little moist red paint in the palm of my hand.

                                         S                                    
      ___________________________________|____                                 
     |                                        NP                              
     |                              __________|____________________            
     |                             NP                              |          
     |                    _________|______________                 |           
     |                   NP                       |                |          
     |        ___________|_______________         |                |           
     |       |          AdjP             |        |                |          
     |       |     ______|____           |        |                |           
     VP      |    |     

In [9]:
def treer(word_list):
    try:
        for tree in parser.parse(word_list):
                tree.pretty_print()
    except ValueError:
        print("No parse tree possible.")
        
def treer2(word_list):
    try:
        trees = list(parser.parse(word_list))
    except ValueError:
        print("No parse tree possible.")
        return None
    return trees

In [54]:
s = "I had a little moist red paint in the palm of my hand.."
trees = treer2(preprocess(s))

In [55]:
nps = np_chunk(trees[0])

In [42]:
len(list(trees[0].subtrees(lambda x: x.label() == "NP")))

6

In [56]:
for phrase in nps:
    phrase.pretty_print()

 NP
 |  
 N 
 |  
 i 

             NP                 
  ___________|_______________    
 |          AdjP             |  
 |     ______|____           |   
 |    |          AdjP        |  
 |    |       ____|____      |   
 |    |      |        AdjP   |  
 |    |      |         |     |   
Det  Adj    Adj       Adj    N  
 |    |      |         |     |   
 a  little moist      red  paint

     NP     
  ___|___    
Det      N  
 |       |   
the     palm

     NP     
  ___|___    
Det      N  
 |       |   
 my     hand

